# <center><span style='color:green'> Household Electricity Consumption </span></center>

## Problem Statement

* Predict the household electricity consumption
* Regression problem

## 1 <span style='color:red'>|</span> Import Libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#for modeling
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score , r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import RandomizedSearchCV

#database
import pymongo

#save the model
import pickle

## 2 <span style='color:red'>|</span> Load Dataset

In [7]:
df = pd.read_csv("household_power_consumption.txt", delimiter=';')
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [8]:
df.shape

(2075259, 9)

* Taking a sample of 60000 as data is too large 

In [9]:
df_sample = df.sample(60000)
df_sample.shape

(60000, 9)

## 3 <span style='color:red'>|</span> Exploratory Data Analysis

In [10]:
df_sample.columns

Index(['Date', 'Time', 'Global_active_power', 'Global_reactive_power',
       'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3'],
      dtype='object')

In [11]:
df_sample.drop(['Date','Time'], axis=1, inplace=True)

In [12]:
df_sample.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
1088346,0.442,0.092,244.080,2.000,0.000,0.000,0.0
1406481,0.382,0.000,238.220,1.600,0.000,0.000,1.0
554150,2.128,0.232,236.950,9.000,0.000,0.000,17.0
833417,0.468,0.360,238.340,2.400,0.000,1.000,1.0
1352261,0.228,0.134,242.190,1.000,0.000,0.000,0.0


In [14]:
df_sample.sample(10)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
1538965,1.404,0.074,242.290,5.800,0.000,1.000,18.0
168529,0.448,0.316,239.640,2.200,0.000,1.000,0.0
775209,0.480,0.000,241.740,2.000,0.000,0.000,1.0
1712688,0.548,0.116,242.880,2.800,0.000,0.000,6.0
706819,1.620,0.214,241.020,6.800,0.000,1.000,18.0
166337,0.406,0.238,240.790,1.800,0.000,2.000,0.0
1508292,0.300,0.086,243.490,1.200,0.000,0.000,1.0
1823180,0.346,0.218,242.690,1.600,0.000,0.000,1.0
1525974,2.516,0.204,237.710,10.600,0.000,0.000,18.0
1938370,0.494,0.178,242.780,2.200,0.000,1.000,1.0


In [17]:
df_sample.dtypes

Global_active_power       object
Global_reactive_power     object
Voltage                   object
Global_intensity          object
Sub_metering_1            object
Sub_metering_2            object
Sub_metering_3           float64
dtype: object